In [1]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [2]:
spark

In [3]:
import myLib

maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache


In [4]:
import example

ModuleNotFoundError: No module named 'example'

In [9]:
sqlContext.sql("use fits_investigation")
numBins=30
sqlContext.sql("""
with rawData as
(
    select 1 as grp, sda_Frequency_hz, spi_index, ra as rightAscension,
    map(

        'dec', sda_declination
    ) as kva,
    map(
        'pixels', raSelectRange
    ) as kvi
    from collatedImages
    distribute by sda_Frequency_hz
    sort by sda_Frequency_hz, spi_index
)
select 
    sda_Frequency_hz as frequency,rightAscension,
    ntile({}) over (sort by sda_Frequency_hz) as bins,
    collect_list(float(a.kva['dec']))as declination
    ,collect_list(array(a.kvi['pixels']))as pixs
from rawData a
group by sda_Frequency_hz, rightAscension
""".format(numBins) ).show(40)

+---------+--------------------+----+--------------------+--------------------+
|frequency|      rightAscension|bins|         declination|                pixs|
+---------+--------------------+----+--------------------+--------------------+
|   1.3765|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0, 0.0, 0.0,...|
|1.3765185|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0, 0.0, 0.0,...|
|1.3765371|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0, 0.0, 0.0,...|
|1.3765556|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0, 0.0, 0.0,...|
| 1.376574|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0, 0.0, 0.0,...|
|1.3765926|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0, 0.0, 0.0,...|
|1.3766111|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0056432555, ...|
|1.3766296|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[-1.20522265E-4...|
|1.3766482|[332.39966, 332.3...|   1|[-46.250008, -46....|[[[0.0017296245, ...|
|1.3766667|[332.39966, 332.3...|   1|[-4

## Pyspark2 command line testing

### The basic dataframe creation test

In [ ]:
pyspark2 \
--py-files 'hdfs:///user/hduser/PythonLibraries/myLib.py' \
--archives 'hdfs:///user/hduser/SharedObjectDLLs/example.zip#example' \
--conf spark.network.timeout=50000 \
--conf spark.executor.heartbeatInterval=240 \
--conf spark.shuffle.service.enabled=True \
--conf spark.sql.shuffle.partitions=30 \
--conf spark.default.parallelism=30 \
--conf spark.dynamicAllocation.enabled=True \
--conf spark.dynamicAllocation.executorIdleTimeout=600 \
--conf spark.network.timeout=800 \
--conf spark.executor.instances=30 \
--conf "spark.executor.extraLibraryPath=:./example" \
--conf "spark.driver.extraLibraryPath=:./example" \
--conf spark.executor.memoryOverhead='4g' \
--conf spark.driver.memoryOverhead='4g' \
--conf spark.rpc.message.maxSize='512' \
--conf spark.scheduler.mode='FAIR' \
--conf spark.kryoserializer.buffer.max='1g' 

spark.sql.codegen.wholeStage=False - to avoid GeneratedIteratorForCOdegenStage grows beyond 64 KB errors 

pyspark2 \
--py-files 'hdfs:///user/hduser/PythonLibraries/myLib.py' \
--archives 'hdfs:///user/hduser/SharedObjectDLLs/example.zip#example' \
--conf spark.network.timeout=50000 \
--conf spark.executor.heartbeatInterval=240 \
--conf spark.shuffle.service.enabled=True \
--conf spark.dynamicAllocation.enabled=True \
--conf spark.maxRemoteBlockSizeFetchToMem='2g' \
--conf spark.dynamicAllocation.executorIdleTimeout=600 \
--conf spark.network.timeout=800 \
--conf spark.sql.codegen.wholeStage=False \
--conf "spark.executor.extraLibraryPath=:./example" \
--conf "spark.driver.extraLibraryPath=:./example" \
--conf spark.executor.memoryOverhead='4g' \
--conf spark.driver.memoryOverhead='4g' \
--conf spark.driver.maxResultSize='4g' \ 
--conf spark.rpc.message.maxSize='512' \
--conf spark.scheduler.mode='FAIR' \
--conf spark.kryoserializer.buffer.max='1g' 

sc.stop()
spark.stop()
SparkSession._instantiatedContext = None

sc=SparkContext(conf=conf)
spark=SparkSession(sc)
sqlContext = SQLContext(sc)
    
import example
import myLib

import numpy as np
from random import randint
import random as rnd
from time import sleep
from pyspark.sql import SQLContext, SparkSession, HiveContext
from pyspark import StorageLevel
from pyspark import SparkFiles
from pyspark.sql import functions as F
import pyspark.sql.functions as f
from pyspark.sql.functions import randn, monotonically_increasing_id, row_number, desc, udf, col, lit
from pyspark.sql.functions import broadcast
from pyspark.sql.types import ArrayType, FloatType,IntegerType, DataType, DoubleType,Row, BooleanType, StringType
from pyspark.sql.types import MapType, StructType,StructField
from myLib import GetDetailArrays,GetSubCube2,GetDataframeSize,CreateFITSSubCubeUDF ,FlattenDataFrame
"""
Set the Spark datatypes
"""
iType=IntegerType()
dType=DoubleType()
fType=FloatType()
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import SparseVector, DenseVector,VectorUDT
from datetime import datetime
import sys
import time as tm
import logging
from time import time, clock

sqlContext.sql("use fits_investigation")
fitsFilename='image.restored.i.SB2338.V2.cube.fits'
raType='RA---SIN'
decType='DEC--SIN'
spectraType='GHz'

numBins=30
raArray, decArray, spectraArray = GetDetailArrays(sqlContext,fitsFilename, raType, decType, spectraType)

hiDec=-46.25
loDec=-47.0
hiRa=332.4
loRa=331.4
loFreq=spectraArray[0]
hiFreq=spectraArray[1000]
CubeSize=[hiRa,loRa,hiDec,loDec,loFreq,hiFreq]
raPix=len( np.array(np.where(np.logical_and(raArray >= loRa, raArray <= hiRa )))[0] )
decPix=len( np.array(np.where(np.logical_and(decArray >= loDec, decArray <= hiDec )))[0] )
frqPix=len( np.array(np.where(np.logical_and(spectraArray >= loFreq, spectraArray <= hiFreq )))[0] )
print(raPix, decPix, frqPix)

size=raPix*decPix*frqPix*32/8e9 # 8e6 = MB, 8e9= GB
print(size)

cacheTempTables=False

subCubeDF,raHeaderIndex,decHeaderIndex,freqHeaderIndex,naxis1, naxis2, naxis4 \
        =GetSubCube2(sqlContext, fitsFilename, decType,spectraType, raArray, \
                     raType, decArray, spectraArray, CubeSize, cacheTempTables)
subCubeDF.createOrReplaceTempView("collatedImages")

compress1=sqlContext.sql("""
with rawData as
(
    select 1 as grp, sda_Frequency_hz, spi_index, ra as rightAscension,
    map(

        'dec', sda_declination
    ) as kva,
    map(
        'pixels', raSelectRange
    ) as kvi
    from collatedImages
    distribute by sda_Frequency_hz
    sort by sda_Frequency_hz, spi_index
)
select 
    sda_Frequency_hz as frequency,rightAscension,
    --ntile({}) over (sort by sda_Frequency_hz) as bins,
    collect_list(float(a.kva['dec']))as declination
    ,collect_list(array(a.kvi['pixels']))as pixs
from rawData a
group by frequency, rightAscension
""".format(numBins))#.cache()

compress1.createOrReplaceTempView("compress_one")

compress2=sqlContext.sql("""
select
    bins,
    rightAscension, declination,
    collect_list(array(float(a.kvi['frequencies']))) as frequencies,
    collect_list(array(a.kva['pixs'])) as pixels
from (
    select
        1 as bins,
        rightAscension, declination,
        map('frequencies', frequency) as kvi,
        map('pixs', pixs) as kva
    from compress_one
    distribute by rightAscension
    sort by frequency
) a
group by bins,
rightAscension, declination
""")

sqlContext.sql("""
    select 1 as grp, sfh_index, sfh_key, sfh_value 
    from sparkfits_fits_headers
    where sfh_fits_file='{}' 
    order by sfh_index
""".format(fitsFilename)
).createOrReplaceTempView("headers")

originalHeader=sqlContext.sql("""
with rawData as
(
    select grp, sfh_index, 
    map(

        'key', sfh_key
    ) as kva,
    map(
        'value', sfh_value
    ) as kvi
    from ( --headers
        select 1 as grp, sfh_index, sfh_key, sfh_value 
        from sparkfits_fits_headers
        where sfh_fits_file='{}' 
        order by sfh_index    
    ) a
    distribute by grp
    sort by grp, sfh_index
)
select 
    grp,
    collect_list(string(a.kva['key']))as keys
    ,collect_list(string(a.kvi['value']))as values
from rawData a
group by grp
""".format(fitsFilename)).select("keys","values")

compress3 = compress2.crossJoin(originalHeader)

compress3 = compress3.repartition("bins")

test1 = udf(CreateFITSSubCubeUDF, StringType() ) 

dfout=compress3.filter(compress3.bins <= 10)\
    .withColumn('process_msg', test1(compress3.rightAscension, \
                                           compress3.declination, \
                                           compress3.frequencies, \
                                           compress3.pixels, \
                                           compress3.keys, \
                                           compress3.values, \
                                           f.lit(fitsFilename))).select('bins','process_msg').persist()   

dfout.show()

In [ ]:
spark2-submit --master yarn --deploy-mode client --py-files 'hdfs:///user/hduser/PythonLibraries/myLib.py' \
--archives 'hdfs:///user/hduser/SharedObjectDLLs/example.zip#example' \
--conf "spark.executor.extraLibraryPath=:./example" \
--conf "spark.driver.extraLibraryPath=:./example" \
DistributedSoFiA_Framework.py

In [ ]:
spark2-submit  \
  --master yarn \
  --name 'Distributed SoFiA' \
  --deploy-mode client \
  --driver-memory 10g \
  --executor-memory 10g \
  --executor-cores 5 \
  --num-executors 23 \
  --py-files 'hdfs:///user/hduser/PythonLibraries/myLib.py' \
  --archives 'hdfs:///user/hduser/SharedObjectDLLs/example.zip#example' \
  --conf "spark.executor.extraLibraryPath=:./example" \
  --conf "spark.driver.extraLibraryPath=:./example" \
  --conf spark.pyspark.python=/home/hduser/.virtualenvs/Elephas/bin/python \
  --conf spark.pyspark.driver.python=/home/hduser/.virtualenvs/Elephas/bin/python \
  --conf spark.shuffle.service.enabled=True \
  --conf spark.sql.shuffle.partitions=23 \
  --conf spark.default.parallelism=23 \
  --conf spark.sql.codegen.wholeStage=False \
  --conf spark.dynamicAllocation.enabled=True \
  --conf spark.executor.memoryOverhead='1g' \
  --conf spark.driver.memoryOverhead='4g' \
  --conf spark.rpc.message.maxSize='512' \
  --conf spark.scheduler.mode='FAIR' \
  --conf spark.kryoserializer.buffer.max='1g' \
  --conf spark.yarn.appMasterEnv.ELEPHAS='True' \
  --conf spark.port.maxRetries=32 \
  --conf spark.driver.maxResultSize='4096m' \
 DistributedSoFiA_Framework.py

https://community.cloudera.com/t5/Community-Articles/Spark-job-fails-with-below-error-when-byte-code-grows-beyond/ta-p/248494
    The codegen wholeStage FALSE directive

spark2-submit  \
  --master yarn \
  --name 'Distributed SoFiA' \
  --deploy-mode client \
  --executor-memory 8g \
  --driver-memory 8g \
  --executor-cores 4 \
  --num-executors 29 \
  --py-files 'hdfs:///user/hduser/PythonLibraries/myLib.py' \
  --archives 'hdfs:///user/hduser/SharedObjectDLLs/example.zip#example' \
  --conf "spark.executor.extraLibraryPath=:./example" \
  --conf "spark.driver.extraLibraryPath=:./example" \
  --conf spark.pyspark.python=/home/hduser/.virtualenvs/Elephas/bin/python \
  --conf spark.pyspark.driver.python=/home/hduser/.virtualenvs/Elephas/bin/python \
  --conf spark.sql.shuffle.partitions=480 \
  --conf spark.default.parallelism=480 \
  --conf spark.shuffle.service.enabled=True \
  --conf spark.sql.codegen.wholeStage=False \
  --conf spark.dynamicAllocation.enabled=True \
  --conf spark.dynamicAllocation.executorIdleTimeout=800 \
  --conf spark.executor.memoryOverhead='4g' \
  --conf spark.driver.memoryOverhead='600m' \
  --conf spark.rpc.message.maxSize='512' \
  --conf spark.scheduler.mode='FAIR' \
  --conf spark.kryoserializer.buffer.max='1g' \
  --conf spark.yarn.appMasterEnv.ELEPHAS='True' \
  --conf spark.port.maxRetries=32 \
  --conf spark.driver.maxResultSize='4096m' \
 DistributedSoFiA_Framework.py > sofia.log 2> sofia_err.log

spark2-submit  \
  --master yarn \
  --deploy-mode client \
  --py-files 'hdfs:///user/hduser/PythonLibraries/myLib.py' \
  --archives 'hdfs:///user/hduser/SharedObjectDLLs/example.zip#example' \
  --conf 'spark.executor.extraLibraryPath=:./example' \
  --conf 'spark.driver.extraLibraryPath=:./example' \
fred.py > fred.log 2> fred_err.log

In [13]:
import math
math.ceil(1185/128)

10

In [8]:
def calculatemod7(val):
    sys.path.append('example')
    import example
    return example.my_mod(val, 7)

In [5]:
sqlContext.sql("use fits_investigation")

DataFrame[]

In [6]:
sqlContext.sql("show partitions sparkfits_images").show(20, False)

+---------------------------------------------------------------+
|partition                                                      |
+---------------------------------------------------------------+
|spi_filename=__HIVE_DEFAULT_PARTITION__/spi_band=1             |
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=0   |
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1   |
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=10  |
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=100 |
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1000|
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1001|
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1002|
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1003|
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1004|
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1005|
|spi_filename=image.restored.i.SB2338.V2.cube.fits/spi_band=1006|
|spi_filen

In [12]:
#import example
import myLib

import numpy as np
from random import randint
import random as rnd
from time import sleep
from pyspark.sql import SQLContext, SparkSession, HiveContext
from pyspark import StorageLevel
from pyspark import SparkFiles
from pyspark.sql import functions as F
import pyspark.sql.functions as f
from pyspark.sql.functions import randn, monotonically_increasing_id, row_number, desc, udf, col, lit
from pyspark.sql.functions import broadcast
from pyspark.sql.types import ArrayType, FloatType,IntegerType, DataType, DoubleType,Row, BooleanType, StringType
from pyspark.sql.types import MapType, StructType,StructField
from myLib import GetDetailArrays,GetSubCube2,GetDataframeSize,CreateFITSSubCubeUDF, FlattenDataFrame 
"""
Set the Spark datatypes
"""
iType=IntegerType()
dType=DoubleType()
fType=FloatType()
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import SparseVector, DenseVector,VectorUDT
from datetime import datetime
import sys
import time as tm
import logging
from time import time, clock

In [7]:
sqlContext.sql("""
    select spi_index from sparkfits_images
    where spi_filename='image.restored.i.SB2338.V2.cube.fits'
    and spi_band=24    
    """).show()

+---------+
|spi_index|
+---------+
|        0|
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       14|
|       15|
|       16|
|       17|
|       18|
|       19|
+---------+
only showing top 20 rows



In [36]:
class Partitioner:
    def __init__(self):
        self.callPerDriverSetup
        
    def callPerDriverSetup(self):
        pass
    
    def callPerPartitionSetup(self):
        sys.path.append('example')                  ### <=== Either append or use the --pyfiles parameter
        import example
         
        self.example=example
        
    def doProcess(self, element):
        ###  here's the call to the C library for each row of the dataframe partition
        ### In here we have to transform the flattened array data to the format SoFiA
        ### requires, as well as the
        return self.example.my_mod(element.spi_index, 7)
        ## return self.example.my_mod(element.wire, 7) 
    
    def processPartition(self, partition):
        self.callPerPartitionSetup()
        for element in partition:
            yield self.doProcess(element)

In [37]:
def main():
    df=sqlContext.sql("""
    select spi_index,spi_filename from sparkfits_images
    where spi_filename='image.restored.i.SB2338.V2.cube.fits'
    and spi_band=24    
    """)
    calcmod7 = udf(calculatemod7, IntegerType())
    dfout=df.withColumn('calc_mod7', calcmod7(df.spi_index)).select('calc_mod7')
    dfout.show()

In [38]:
def main():
    df=sqlContext.sql("""
    select spi_index from sparkfits_images
    where spi_filename='image.restored.i.SB2338.V2.cube.fits'
    and spi_band=24    
    """)
    p=Partitioner()
    rddout=df.rdd.mapPartitions(p.processPartition)
    print("Done!")
    rddout.take(20)
    print("Fred!")
    return rddout


In [ ]:
def main():
    fitsFilename='image.restored.i.SB2338.V2.cube.fits'
    raType='RA---SIN'
    decType='DEC--SIN'
    spectraType='Hz'
    
    raArray, decArray, spectraArray = GetDetailArrays(sqlContext, fitsFilename, raType, decType, spectraType)
    
    raBucketSize=15
    decBucketSize=15
    freqBucketSize=15
    
    raHist=np.histogram(raArray, raBucketSize)
    decHist=np.histogram(np.flip(decArray), decBucketSize)
    freqHist=np.histogram(spectraArray, freqBucketSize)
    arr = np.empty((0,6), float)
    
    for i in np.arange(raBucketSize):
        for j in  np.arange(decBucketSize):
            for k in np.arange(freqBucketSize):
                x=np.array([[raHist[1][i], raHist[1][i+1], decHist[1][j], decHist[1][j+1], freqHist[1][k], freqHist[1][k+1] ]])
                arr=np.append(arr, x, axis=0)
                pass
            pass
        pass
    hiDec=np.flip(decHist[1])[0]
    loDec=np.flip(decHist[1])[1]
    hiRa=raHist[1][1]
    loRa=raHist[1][0]
    loFreq=freqHist[1][0] #spectraArray[0]
    hiFreq=freqHist[1][1] #spectraArray[299]
    
    depthOfCubes=5
    
    CubeSize=[hiRa,loRa,hiDec,loDec,loFreq,hiFreq]
    
    frqPix=len( np.array(np.where(np.logical_and(spectraArray >= loFreq, spectraArray <= hiFreq )))[0] )
    
    ntileCount=frqPix/depthOfCubes
    
    subCubeDF,raHeaderIndex,decHeaderIndex,freqHeaderIndex,naxis1, naxis2, naxis4 \
    =GetSubCube2(\
                 sqlContext, fitsFilename, decType,spectraType, \
                 raArray, raType, decArray, spectraArray, CubeSize, round(ntileCount) )
    
    compress3=FlattenDataFrame(sqlContext, subCubeDF, fitsFilename)
    
    p=Partitioner()
    
    rddout=compress3.rdd.mapPartitions(p.processPartition)
    
    




    

In [39]:
if __name__ == '__main__':
    x=main()

Done!
Fred!


In [40]:
x.take(80)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2]

In [3]:
sqlContext

In [4]:
import sys


In [5]:
sc.addPyFile('hdfs:///user/hduser/PythonLibraries/myLib.py')

In [6]:
sc.addFile('hdfs:///user/hduser/SharedObjectDLLs/example.zip')

In [7]:
from pyspark import SparkFiles
zipPath=SparkFiles.get("example.zip"),

In [8]:
zipPath[0]

'/mnt/FITs/Spark/tmp/spark-e31a47db-6cd6-4bfc-a9d1-b952664b0043/userFiles-c6c3a740-b147-451b-a672-5d41f688793e/example.zip'

In [9]:
sys.path.append(zipPath[0])

In [10]:
import myLib as m

maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache


In [11]:
import example

ModuleNotFoundError: No module named '_example'

In [ ]:
sqlContext

In [ ]:
sqlContext.sql("use fits_investigation")

In [ ]:
df=sqlContext.sql("select * from sparkfits_images where spi_filename='ASKAPsim_pol1.image.I.restored.fits' and spi_band=12")

In [ ]:
x=m.GetDataframeSize(sc, df)

In [ ]:
x